# Tema: Analíse do fluxo aéreo com a mudança dos aeroportos de Natal.

1 - Faremos a importação das bibliotecas necessárias para manipulação de arquivos CSV e criação e manipulação de mapas.

In [0]:
!pip install folium==0.8.2
import pandas as pd
import numpy as np
import folium
import json
from folium import plugins

In [0]:
folium.__version__
#http://www.anac.gov.br/dadosabertos

'0.8.2'

In [0]:
# install the latest version of requests
!pip install requests==v2.21.0

In [0]:
mapa_brazil = folium.Map(location=[-11.804592,-45.6329569],zoom_start=4.79
                         ,tiles='Stamen Terrain')
mapa_brazil

In [0]:
aeroportos_publicos =pd.read_csv('https://drive.google.com/uc?authuser=0&id=1xHhzSTRDr6apDqeE5WWElet09HJ-msxj&export=download',error_bad_lines=False, encoding="latin-1",sep=';')
aeroportos_publicos = aeroportos_publicos.drop(aeroportos_publicos.columns[7:], axis=1)
aeroportos_publicos = aeroportos_publicos
aeroportos_publicos.columns = aeroportos_publicos.iloc[1]
aeroportos_publicos= aeroportos_publicos.reindex(aeroportos_publicos.index.drop(0))
aeroportos_publicos= aeroportos_publicos.reindex(aeroportos_publicos.index.drop(1))
aeroportos_publicos = aeroportos_publicos.reset_index(drop=True)
aeroportos_publicos.head()

1,CÓDIGO OACI,CIAD,NOME,MUNICÍPIO ATENDIDO,UF,LATITUDE,LONGITUDE
0,SBAA,PA0008,CONCEIÇÃO DO ARAGUAIA,CONCEIÇÃO DO ARAGUAIA,PA,8° 20' 55'' S,49° 18' 11'' W
1,SBAE,SP0010,BAURU/AREALVA,BAURU E AREALVA,SP,22° 9' 28'' S,49° 4' 6'' W
2,SBAQ,SP0012,BARTOLOMEU DE GUSMÃO,ARARAQUARA,SP,21° 48' 16'' S,48° 8' 25'' W
3,SBAR,SE0001,SANTA MARIA,ARACAJU,SE,10° 59' 7'' S,37° 4' 24'' W
4,SBAT,MT0003,PILOTO OSVALDO MARQUES DIAS,ALTA FLORESTA,MT,9° 51' 59'' S,56° 6' 18'' W


In [0]:
todos_estados_sem_nordeste=['AC','AM','AP','PA','RO','RR','TO','GO','MS','MT','ES','MG','RJ','SP','PR','RS','SC','DF']

#dropando estados diferentes do nordeste
def retirar_estados(df):
  for estado in todos_estados_sem_nordeste:
    aeroportos_publicos.drop(aeroportos_publicos[aeroportos_publicos.UF == estado].index,inplace = True)
retirar_estados(aeroportos_publicos)
aeroportos_publicos = aeroportos_publicos.reset_index(drop=True)
aeroportos_publicos.head()

1,CÓDIGO OACI,CIAD,NOME,MUNICÍPIO ATENDIDO,UF,LATITUDE,LONGITUDE
0,SBAR,SE0001,SANTA MARIA,ARACAJU,SE,10° 59' 7'' S,37° 4' 24'' W
1,SBCI,MA0003,BRIGADEIRO LYSIAS AUGUSTO RODRIGUES,CAROLINA,MA,7° 19' 14'' S,47° 27' 31'' W
2,SSCV,BA0012,CARAVELAS,CARAVELAS,BA,17° 39' 2'' S,39° 15' 16'' W
3,SBFN,PE0003,FERNANDO DE NORONHA,FERNANDO DE NORONHA,PE,3° 51' 17'' S,32° 25' 42'' W
4,SBFZ,CE0001,PINTO MARTINS,FORTALEZA,CE,3° 46' 33'' S,38° 31' 56'' W


In [0]:
aeroportos_publicos.LATITUDE = (aeroportos_publicos.LATITUDE.str.split(r'[°\'"]', expand=True)).iloc[:, :3].astype('float').dot([1, 1./60, 1./3600])*-1

In [0]:
aeroportos_publicos.LONGITUDE = aeroportos_publicos.LONGITUDE.str.split(r'[°\'"]', expand=True).iloc[:, :3].astype('float').dot([1, 1./60, 1./3600])*-1

In [0]:
aeroportos_publicos[aeroportos_publicos['UF']=='RN']

1,CÓDIGO OACI,CIAD,NOME,MUNICÍPIO ATENDIDO,UF,LATITUDE,LONGITUDE
13,SBMS,RN0002,DIX-SEPT ROSADO,MOSSORÓ,RN,-5.195833,-37.361667
19,SBSG,RN0001,GOVERNADOR ALUIZIO ALVES,SÃO GONÇALO DO AMARANTE,RN,-5.768889,-35.366389
77,SNKK,RN0005,CAICÓ,CAICÓ,RN,-6.439167,-37.078056
78,SNKN,RN0004,CURRAIS NOVOS,CURRAIS NOVOS,RN,-6.280833,-36.540556
112,SNUC,RN0003,AÇU,AÇU,RN,-5.595000,-36.960833


In [0]:
import requests
# configure a generical header
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

In [0]:
#Criando mapa com zoom no nordeste brasileiro
mapa_nordeste = folium.Map(location=[-8.8864889,-39.6786708],zoom_start=5.3,
                           tiles='Stamen Terrain')

In [0]:
estado_jsons = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/2/?formato=application/vnd.geo+json&resolucao=2",
                           headers=headers)
estado_json = estado_jsons.json()
estado_json2 = estado_json
folium.GeoJson(estado_json).add_to(mapa_nordeste)

In [0]:
mapa_nordeste

In [0]:
aeroportos_publicos.head()

1,CÓDIGO OACI,CIAD,NOME,MUNICÍPIO ATENDIDO,UF,LATITUDE,LONGITUDE
0,SBAR,SE0001,SANTA MARIA,ARACAJU,SE,-10.985278,-37.073333
1,SBCI,MA0003,BRIGADEIRO LYSIAS AUGUSTO RODRIGUES,CAROLINA,MA,-7.320556,-47.458611
2,SSCV,BA0012,CARAVELAS,CARAVELAS,BA,-17.650556,-39.254444
3,SBFN,PE0003,FERNANDO DE NORONHA,FERNANDO DE NORONHA,PE,-3.854722,-32.428333
4,SBFZ,CE0001,PINTO MARTINS,FORTALEZA,CE,-3.775833,-38.532222


In [0]:
tiles = ['openstreetmap','Mapbox Bright','Mapbox Control Room',
        'stamenterrain','stamenterrain','stamentoner',
        'stamenwatercolor','cartodbpositron','cartodbdark_matter']
for tiles_ in tiles: 
  folium.TileLayer(tiles_).add_to(mapa_nordeste)
folium.LayerControl().add_to(mapa_nordeste)

In [0]:
mapa_nordeste

In [0]:
for counter in range(134):
  folium.Marker([aeroportos_publicos.LATITUDE[counter],aeroportos_publicos.LONGITUDE[counter]], 
               popup='<ul><li><h5>'+aeroportos_publicos.NOME[counter]+'</li></h5><ul><li>'
                +aeroportos_publicos['UF'][counter]+'</li><li>'+aeroportos_publicos['MUNICÍPIO ATENDIDO'][counter]+'</ul></li></ul>',
               tooltip=aeroportos_publicos['CÓDIGO OACI'][counter],
               icon=folium.Icon(color='blue',
                    icon_color='black',
                    icon='plane',
                    prefix='fa')).add_to(mapa_nordeste)

In [0]:
mapa_nordeste

In [0]:
#cluster_inutil
data = np.array(
    [
        aeroportos_publicos['LATITUDE'],aeroportos_publicos['LONGITUDE']
    ]
).T

# Create a map
m = folium.Map(
    location=[-8.8864889,-39.6786708],zoom_start=5.3,
    tiles='Stamen Terrain'
)

plugins.MarkerCluster(data).add_to(m)
estado_jsons = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/2/?formato=application/vnd.geo+json&resolucao=2",
                           headers=headers)
estado_json = estado_jsons.json()
folium.GeoJson(estado_json).add_to(m)
m

# Analisando Dados de Emigração e Imigração de Aeroportos do Nordeste Brasileiro (2013)

In [0]:
#Alagoas(1), Bahia(2), Ceará(1), Maranhão (3) , Paraiba(2), Pernambuco(2), Piaui(2), RN(1), Sergipe(1)
aeroportos_fed_ne_list = ['SBMO',
                          'SBIL','SBUF',
                          'SBJU',
                          'SBIZ', 'SBSL', 'SNCW',
                          'SBJP', 'SBKG',
                          'SBPL','SBRF',
                          'SBPB','SBTE',
                          'SBNT','SBSG',
                          'SBAR']

aeroportos_fed_ne_dict = {'AL':['SBMO'],
                          'BA':['SBIL','SBUF'],
                          'CE':['SBJU'],
                          'MA':['SBIZ', 'SBSL', 'SNCW'],
                          'PB':['SBJP', 'SBKG'],
                          'PE':['SBPL','SBRF'],
                          'PI':['SBPB','SBTE'],
                          'RN':['SBNT','SBSG'],
                          'SE':['SBAR']}

In [0]:
#funções para limpar os dataframes: apenas colunas uteis para estudo
def clean_df_2013(df_mes):
  df_mes = pd.DataFrame(df_mes, columns=['Aeroporto Origem',	'Aeroporto Destino', 'Situação'])
  df_mes.columns = ['aeroporto_origem', 'aeroporto_destino', 'situacao']
  df_mes =  df_mes[(df_mes.situacao == 'Realizado')]
  df_mes.reset_index(drop=True)
  df_mes.head()
  return df_mes

In [0]:
url_fixo_2013 = 'http://www.anac.gov.br/assuntos/dados-e-estatisticas/base-historica-1/vra/2013/vra_do_mes_'
urls_csv_2013 = ['2013_01.zip', '2013_02.zip', '2013_03.zip',
                '2013_04.zip', '2013_05.zip', '2013_06.zip', 
                '2013_07.zip', '2013_08.zip', '2013_09.zip', 
                '2013_10.zip', '2013_11.csv', '2013_12.zip']  

In [0]:
frames = []
for url_csv in urls_csv_2013:
  if(url_csv == '2013_11.csv'):
    df = clean_df_2013(pd.read_csv( url_fixo_2013 + url_csv ,error_bad_lines=False, encoding="latin-1",sep=';'))
  else:
    df = clean_df_2013(pd.read_csv(url_fixo_2013 + url_csv, compression='zip', header=0, error_bad_lines=False, encoding="latin-1", sep=';', quotechar='"'))
  frames.append(df)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [0]:
#Concatenando todos os dataframes do ano 2013
result_df_2013 = pd.concat(frames, ignore_index=True)
result_df_2013.info()
result_df_2013.loc[1,'aeroporto_origem']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1049188 entries, 0 to 1049187
Data columns (total 3 columns):
aeroporto_origem     1049188 non-null object
aeroporto_destino    1049188 non-null object
situacao             1049188 non-null object
dtypes: object(3)
memory usage: 24.0+ MB


'SBMK'

In [0]:
#Contando quantidade de voos que originam de aeroportos federais do nordeste
voos_origem_estados_ne = {}

voos_destino_estados_ne = {}

for estado,aeroportos in aeroportos_fed_ne_dict.items():
  voos_origem_estados_ne[estado] = 0
  voos_destino_estados_ne[estado] = 0
  for aero in aeroportos:
    voos_origem_estados_ne[estado] += result_df_2013[(result_df_2013.aeroporto_origem == aero)].count()[0]
    voos_destino_estados_ne[estado] += result_df_2013[(result_df_2013.aeroporto_destino == aero)].count()[0]

In [0]:
#Transformando o dicionario com dados sobre emigração dos aeroportos do nordeste em dataframe
voos_origem_ne_df = pd.DataFrame.from_dict({"QTD_VOOS_ORIGEM":voos_origem_estados_ne})
voos_origem_ne_df.index.name = "SIGLA"
voos_origem_ne_df

,QTD_VOOS_ORIGEM
SIGLA,
AL,8310
BA,3404
CE,2480
MA,10018
PB,5693
PE,32787
PI,4680
RN,9232
SE,6967


In [0]:
#Transformando o dicionario com dados sobre imigração dos aeroportos do nordeste em dataframe
voos_destino_ne_df = pd.DataFrame.from_dict({'QTD_VOOS_DESTINO':voos_destino_estados_ne})
voos_destino_ne_df.index.name = "SIGLA"

voos_destino_ne_df

,QTD_VOOS_DESTINO
SIGLA,
AL,8292
BA,3396
CE,2478
MA,10040
PB,5689
PE,32780
PI,4679
RN,9226
SE,6935


In [0]:
states = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/regioes/2/estados",
                       headers=headers).json()

states_northeast = [estados["nome"] for estados in states]
states_id = [estados["id"] for estados in states]
state_initials = [estados["sigla"] for estados in states]
data_northeast = pd.DataFrame.from_dict({'ID':states_id,'ESTADO':states_northeast,"SIGLA":state_initials})


In [0]:
data_northeast

,ID,ESTADO,SIGLA
0,21,Maranhão,MA
1,22,Piauí,PI
2,23,Ceará,CE
3,24,Rio Grande do Norte,RN
4,25,Paraíba,PB
5,26,Pernambuco,PE
6,27,Alagoas,AL
7,28,Sergipe,SE
8,29,Bahia,BA


In [0]:
estado_json

{'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4674'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [[[-41.3226, -2.9213],
      [-41.3207, -2.9564],
      [-41.2984, -2.9684],
      [-41.2712, -2.9716],
      [-41.2489, -3.0304],
      [-41.262, -3.0506],
      [-41.2566, -3.088],
      [-41.3113, -3.1363],
      [-41.328, -3.1602],
      [-41.3224, -3.1797],
      [-41.3958, -3.304],
      [-41.3979, -3.3275],
      [-41.4123, -3.3406],
      [-41.4215, -3.3705],
      [-41.4057, -3.3625],
      [-41.3985, -3.3882],
      [-41.3874, -3.3891],
      [-41.3869, -3.4092],
      [-41.3467, -3.3855],
      [-41.324, -3.3859],
      [-41.3269, -3.4463],
      [-41.3197, -3.484],
      [-41.303, -3.4921],
      [-41.326, -3.5082],
      [-41.3256, -3.5364],
      [-41.3368, -3.5538],
      [-41.3646, -3.5579],
      [-41.368, -3.5785],
      [-41.3439, -3.5904],
      [-41.3313, -3.6246],
      [-41.3417, -3.6807],
      [-41.3076, -3.6848],
      [-41.2793, -3.7007],

In [0]:
final_data_2013 = pd.merge(left = data_northeast,right = voos_origem_ne_df,on="SIGLA",how= "outer")
final_data_2013.set_index('ID', inplace=True)
final_data_2013

,ESTADO,SIGLA,QTD_VOOS_ORIGEM
ID,,,
21,Maranhão,MA,10018
22,Piauí,PI,4680
23,Ceará,CE,2480
24,Rio Grande do Norte,RN,9232
25,Paraíba,PB,5693
26,Pernambuco,PE,32787
27,Alagoas,AL,8310
28,Sergipe,SE,6967
29,Bahia,BA,3404


In [0]:
for state in estado_json['features']: 
  codarea = state['properties']['codarea']
  state['properties']['sigla'] = str(final_data_2013.loc[int(codarea), "SIGLA"])
  state['properties']['qtd_voos'] = str(final_data_2013.loc[int(codarea), "QTD_VOOS_ORIGEM"])

In [0]:
estado_json

{'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4674'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [[[-41.3226, -2.9213],
      [-41.3207, -2.9564],
      [-41.2984, -2.9684],
      [-41.2712, -2.9716],
      [-41.2489, -3.0304],
      [-41.262, -3.0506],
      [-41.2566, -3.088],
      [-41.3113, -3.1363],
      [-41.328, -3.1602],
      [-41.3224, -3.1797],
      [-41.3958, -3.304],
      [-41.3979, -3.3275],
      [-41.4123, -3.3406],
      [-41.4215, -3.3705],
      [-41.4057, -3.3625],
      [-41.3985, -3.3882],
      [-41.3874, -3.3891],
      [-41.3869, -3.4092],
      [-41.3467, -3.3855],
      [-41.324, -3.3859],
      [-41.3269, -3.4463],
      [-41.3197, -3.484],
      [-41.303, -3.4921],
      [-41.326, -3.5082],
      [-41.3256, -3.5364],
      [-41.3368, -3.5538],
      [-41.3646, -3.5579],
      [-41.368, -3.5785],
      [-41.3439, -3.5904],
      [-41.3313, -3.6246],
      [-41.3417, -3.6807],
      [-41.3076, -3.6848],
      [-41.2793, -3.7007],

In [0]:
#Criando Choropleth 2013 Imigração
k = folium.Map(
    location=[-8.8864889,-39.6786708],zoom_start=5.3,
    tiles='Stamen Terrain'
)

bins = np.linspace(final_data_2013['QTD_VOOS_ORIGEM'].min(),
                   final_data_2013['QTD_VOOS_ORIGEM'].max(),
                   10).tolist()


folium.Choropleth(
    geo_data=estado_json,
    data=final_data_2013,
    name= "ID",
    columns=['SIGLA', 'QTD_VOOS_ORIGEM'],
    key_on='feature.properties.sigla',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='#Voos Saindo do Nordeste',
    bins=bins
).add_to(k)

folium.LayerControl().add_to(k)

k

In [0]:
estado_json

{'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4674'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [[[-41.3226, -2.9213],
      [-41.3207, -2.9564],
      [-41.2984, -2.9684],
      [-41.2712, -2.9716],
      [-41.2489, -3.0304],
      [-41.262, -3.0506],
      [-41.2566, -3.088],
      [-41.3113, -3.1363],
      [-41.328, -3.1602],
      [-41.3224, -3.1797],
      [-41.3958, -3.304],
      [-41.3979, -3.3275],
      [-41.4123, -3.3406],
      [-41.4215, -3.3705],
      [-41.4057, -3.3625],
      [-41.3985, -3.3882],
      [-41.3874, -3.3891],
      [-41.3869, -3.4092],
      [-41.3467, -3.3855],
      [-41.324, -3.3859],
      [-41.3269, -3.4463],
      [-41.3197, -3.484],
      [-41.303, -3.4921],
      [-41.326, -3.5082],
      [-41.3256, -3.5364],
      [-41.3368, -3.5538],
      [-41.3646, -3.5579],
      [-41.368, -3.5785],
      [-41.3439, -3.5904],
      [-41.3313, -3.6246],
      [-41.3417, -3.6807],
      [-41.3076, -3.6848],
      [-41.2793, -3.7007],

In [0]:
final_data_2013_chegada = pd.merge(left = data_northeast,right = voos_destino_ne_df,on="SIGLA",how= "outer")
final_data_2013_chegada.set_index('ID', inplace=True)
final_data_2013_chegada

,ESTADO,SIGLA,QTD_VOOS_DESTINO
ID,,,
21,Maranhão,MA,10040
22,Piauí,PI,4679
23,Ceará,CE,2478
24,Rio Grande do Norte,RN,9226
25,Paraíba,PB,5689
26,Pernambuco,PE,32780
27,Alagoas,AL,8292
28,Sergipe,SE,6935
29,Bahia,BA,3396


In [0]:
for state in estado_json2['features']: 
  codarea = state['properties']['codarea']
  state['properties']['sigla'] = str(final_data_2013_chegada.loc[int(codarea), "SIGLA"])
  state['properties']['qtd_voos'] = str(final_data_2013_chegada.loc[int(codarea), "QTD_VOOS_DESTINO"])

In [0]:
#Criando Choropleth 2013 Emigração
h = folium.Map(
    location=[-8.8864889,-39.6786708],zoom_start=5.3,
    tiles='Stamen Terrain'
)

bins = np.linspace(final_data_2013_chegada['QTD_VOOS_DESTINO'].min(),
                   final_data_2013_chegada['QTD_VOOS_DESTINO'].max(),
                   10).tolist()


folium.Choropleth(
    geo_data=estado_json2,
    data=final_data_2013_chegada,
    name= "ID",
    columns=['SIGLA', 'QTD_VOOS_DESTINO'],
    key_on='feature.properties.sigla',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='#Voos Chegando no Nordeste',
    bins=bins
).add_to(h)

folium.LayerControl().add_to(h)

k

# Analisando Dados de Emigração e Imigração de Aeroportos do Nordeste Brasileiro (2018)

In [0]:
#Pegar lista aeroportos do nordeste a partir da tabela de aeroportos publicos
estados_ne_2018_dict = {}

siglas_ne_list = list(aeroportos_fed_ne_dict)

for sigla_estado in siglas_ne_list:
  estados_ne_2018_dict[sigla_estado] = aeroportos_publicos[aeroportos_publicos['UF']==sigla_estado]['CÓDIGO OACI']

estados_ne_2018_dict['AL']

12    SBMO
42    SNAL
92    SNPE
Name: CÓDIGO OACI, dtype: object

In [0]:
estados_ne_2018_dict['BA']

2      SSCV
5      SBIL
10     SBLE
11     SBLP
16     SBPS
17     SBQV
21     SBSV
23     SBUF
24     SDFX
25     SDKE
26     SDKF
27     SDKJ
28     SDLE
29     SDLI
30     SDLK
47     SNBL
48     SNBR
50     SNBX
51     SNBZ
54     SNDW
55     SNED
56     SNEU
60     SNGI
62     SNGT
64     SNIB
65     SNIC
66     SNIE
68     SNIO
69     SNIP
70     SNIT
       ... 
81     SNKS
82     SNKU
84     SNMC
85     SNMJ
86     SNMR
87     SNNH
89     SNOC
93     SNPI
97     SNQU
99     SNRD
100    SNRM
104    SNSE
106    SNST
107    SNTF
108    SNTQ
109    SNTR
111    SNTY
113    SNUT
114    SNVB
115    SNVD
116    SNVV
119    SNXQ
121    SNYT
122    SNZW
124    SSLS
125    SSQP
126    SSQZ
127    SSRF
128    SSRK
130    SSVC
Name: CÓDIGO OACI, Length: 66, dtype: object

In [0]:
aeroportos_rn = aeroportos_publicos[aeroportos_publicos['UF']=='RN']['CÓDIGO OACI']
aeroportos_rn

13     SBMS
19     SBSG
77     SNKK
78     SNKN
112    SNUC
Name: CÓDIGO OACI, dtype: object

In [0]:
#Pegando a lista de siglas dos estado brasileiros
ne_list = list(aeroportos_fed_ne_dict)

#Contando quantidade de voos que originam de aeroportos federais do nordeste

voos_origem_estados_ne = {}

voos_destino_estados_ne = {}

for estado,aeroportos in aeroportos_fed_ne_dict.items():
  voos_origem_estados_ne[estado] = 0
  voos_destino_estados_ne[estado] = 0
  for aero in aeroportos:
    voos_origem_estados_ne[estado] += result_df_2013[(result_df_2013.aeroporto_origem == aero)].count()[0]
    voos_destino_estados_ne[estado] += result_df_2013[(result_df_2013.aeroporto_destino == aero)].count()[0]

In [0]:
def clean_df_2018(df_mes):
  df_mes = pd.DataFrame(df_mes, columns=['ICAO Aeródromo Origem',	'ICAO Aeródromo Destino', 'Situação Voo'])
  df_mes.columns = ['aeroporto_origem', 'aeroporto_destino', 'situacao']
  df_mes =  df_mes[(df_mes.situacao == 'REALIZADO')]
  df_mes.reset_index(drop=True)
  return df_mes

In [0]:
#Caminhos para tabelas que contém informações do histórico de voo em 2018
url_fixo_2018 = 'http://www.anac.gov.br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra'
urls_csv_2018 = ['-01_2018.csv', '-02_2018.csv', '_032018.csv',
                '_04.csv', '_05.csv', '_062018.csv', 
                '_072018.csv', '_082018.csv', '_092018.csv', 
                 '_102018.csv', '_112018.csv', '_122018.csv'] 


In [0]:
frames2018 = []
for url_csv in urls_csv_2018:
  df = clean_df_2018(pd.read_csv( url_fixo_2018 + url_csv ,error_bad_lines=False, encoding="latin-1",sep=';'))
  frames2018.append(df)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df = clean_df_2018(pd.read_csv( 'http://www.anac.gov.br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra-01_2018.csv' ,error_bad_lines=False, encoding="latin-1",sep=';'))
df.head()

,aeroporto_origem,aeroporto_destino,situacao
0,KMIA,SBEG,REALIZADO
1,KMIA,SBEG,REALIZADO
2,KMIA,SBEG,REALIZADO
3,KMIA,SBEG,REALIZADO
4,KMIA,SBEG,REALIZADO


In [0]:
result_df_2018 = pd.concat(frames2018, ignore_index=True)
result_df_2018.info()
result_df_2018.loc[1,'aeroporto_origem']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988725 entries, 0 to 988724
Data columns (total 3 columns):
aeroporto_origem     988725 non-null object
aeroporto_destino    988725 non-null object
situacao             988725 non-null object
dtypes: object(3)
memory usage: 22.6+ MB


'KMIA'

In [0]:
#Contando quantidade de voos que originam de aeroportos federais do nordeste
voos_origem_estados_ne_2018 = {}

voos_destino_estados_ne_2018 = {}

for estado,aeroportos in aeroportos_fed_ne_dict.items():
  voos_origem_estados_ne_2018[estado] = 0
  voos_destino_estados_ne_2018[estado] = 0
  for aero in aeroportos:
    voos_origem_estados_ne_2018[estado] += result_df_2018[(result_df_2018.aeroporto_origem == aero)].count()[0]
    voos_destino_estados_ne_2018[estado] += result_df_2018[(result_df_2018.aeroporto_destino == aero)].count()[0]

In [0]:
#Transformando o dicionario com dados sobre emigração dos aeroportos do nordeste em dataframe
voos_origem_ne_df_2018 = pd.DataFrame.from_dict({"QTD_VOOS_ORIGEM":voos_origem_estados_ne_2018})
voos_origem_ne_df_2018.index.name = "SIGLA"
voos_origem_ne_df_2018

,QTD_VOOS_ORIGEM
SIGLA,
AL,7750
BA,2928
CE,2890
MA,8010
PB,6216
PE,36103
PI,4317
RN,8886
SE,4557


In [0]:
#Transformando o dicionario com dados sobre imigração dos aeroportos do nordeste em dataframe
voos_destino_ne_df_2018 = pd.DataFrame.from_dict({'QTD_VOOS_DESTINO':voos_destino_estados_ne_2018})
voos_destino_ne_df_2018.index.name = "SIGLA"
voos_destino_ne_df_2018

,QTD_VOOS_DESTINO
SIGLA,
AL,7760
BA,2932
CE,2888
MA,7999
PB,6210
PE,36020
PI,4305
RN,8933
SE,4554


In [0]:
final_data_2018 = pd.merge(left = data_northeast,right = voos_origem_ne_df_2018,on="SIGLA",how= "outer")
final_data_2018.set_index('ID', inplace=True)
final_data_2018

,ESTADO,SIGLA,QTD_VOOS_ORIGEM
ID,,,
21,Maranhão,MA,8010
22,Piauí,PI,4317
23,Ceará,CE,2890
24,Rio Grande do Norte,RN,8886
25,Paraíba,PB,6216
26,Pernambuco,PE,36103
27,Alagoas,AL,7750
28,Sergipe,SE,4557
29,Bahia,BA,2928


In [0]:
final_data_2018_chegada = pd.merge(left = data_northeast,right = voos_destino_ne_df_2018,on="SIGLA",how= "outer")
final_data_2018_chegada.set_index('ID', inplace=True)
final_data_2018_chegada

,ESTADO,SIGLA,QTD_VOOS_DESTINO
ID,,,
21,Maranhão,MA,7999
22,Piauí,PI,4305
23,Ceará,CE,2888
24,Rio Grande do Norte,RN,8933
25,Paraíba,PB,6210
26,Pernambuco,PE,36020
27,Alagoas,AL,7760
28,Sergipe,SE,4554
29,Bahia,BA,2932


In [0]:
for state in estado_json2['features']: 
  codarea = state['properties']['codarea']
  state['properties']['sigla'] = str(final_data_2018.loc[int(codarea), "SIGLA"])
  state['properties']['qtd_voos'] = str(final_data_2018.loc[int(codarea), "QTD_VOOS_ORIGEM"])

In [0]:
for state in estado_json2['features']: 
  codarea = state['properties']['codarea']
  state['properties']['sigla'] = str(final_data_2018_chegada.loc[int(codarea), "SIGLA"])
  state['properties']['qtd_voos'] = str(final_data_2018_chegada.loc[int(codarea), "QTD_VOOS_DESTINO"])

In [0]:
#Criando Choropleth 2018 Imigração
z = folium.Map(
    location=[-8.8864889,-39.6786708],zoom_start=5.3,
    tiles='Stamen Terrain'
)

bins = np.linspace(final_data_2018['QTD_VOOS_ORIGEM'].min(),
                   final_data_2018['QTD_VOOS_ORIGEM'].max(),
                   10).tolist()


folium.Choropleth(
    geo_data=estado_json,
    data=final_data_2018,
    name= "ID",
    columns=['SIGLA', 'QTD_VOOS_ORIGEM'],
    key_on='feature.properties.sigla',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='#Voos Saindo do Nordeste 2018',
    bins=bins
).add_to(z)

folium.LayerControl().add_to(z)

z

In [0]:
#Criando Choropleth 2013 Emigração
u = folium.Map(
    location=[-8.8864889,-39.6786708],zoom_start=5.3,
    tiles='Stamen Terrain'
)

bins = np.linspace(final_data_2018_chegada['QTD_VOOS_DESTINO'].min(),
                   final_data_2018_chegada['QTD_VOOS_DESTINO'].max(),
                   10).tolist()


folium.Choropleth(
    geo_data=estado_json2,
    data=final_data_2018_chegada,
    name= "ID",
    columns=['SIGLA', 'QTD_VOOS_DESTINO'],
    key_on='feature.properties.sigla',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='#Voos Chegando no Nordeste',
    bins=bins
).add_to(u)

folium.LayerControl().add_to(u)

u